In [18]:
import pandas as pd
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import optuna

from giskard import Dataset, Model, scan, testing
import pickle

from minio import Minio
from minio.error import S3Error

import clickhouse_connect

import mlflow 
import mlflow.catboost
import mlflow.sklearn
import mlflow.data
from mlflow.models import infer_signature

from functools import reduce
import warnings
from datetime import datetime

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

SPARK_COMPAT_VERSION = os.getenv('SPARK_COMPAT_VERSION')
SCALA_COMPAT_VERSION = os.getenv('SCALA_COMPAT_VERSION')
CATBOOST_SPARK_VERSION = os.getenv('CATBOOST_SPARK_VERSION')
CLICKHOUSE_HOST = os.getenv('CLICKHOUSE_HOST')
CLICKHOUSE_PORT = os.getenv('CLICKHOUSE_PORT')
CLICKHOUSE_USER = os.getenv('CLICKHOUSE_USER')
CLICKHOUSE_PASSWORD = os.getenv('CLICKHOUSE_PASSWORD')
ACCESS_KEY=os.getenv('MINIO_ACCESS_KEY')
SECRET_KEY=os.getenv('MINIO_SECRET_KEY')

In [19]:
COLUMN_TYPES = {
    'age': 'category',
    'sex': 'category',
    'job': 'category',
    'housing': 'category',
    'credit_amount': 'numeric',
    'duration': 'numeric'
}

TARGET_COLUMN_NAME = 'default'
FEATURE_COLUMNS = [i for i in COLUMN_TYPES.keys()]
FEATURE_TYPES = {i: COLUMN_TYPES[i] for i in COLUMN_TYPES if i != TARGET_COLUMN_NAME}

COLUMNS_TO_SCALE = [key for key in COLUMN_TYPES.keys() if COLUMN_TYPES[key] == "numeric"]
COLUMNS_TO_ENCODE = [key for key in COLUMN_TYPES.keys() if COLUMN_TYPES[key] == "category"]

In [5]:
client = clickhouse_connect.get_client(host = 'localhost', 
                                       port = CLICKHOUSE_PORT, 
                                       user = CLICKHOUSE_USER, 
                                       password = CLICKHOUSE_PASSWORD)

# Get data

In [12]:
job_list = {
    0: 'unskilled and non-resident', 
    1: 'unskilled and resident', 
    2: 'skilled', 
    3: 'highly skilled'
}

query = fr'''
select {reduce(lambda a,b: a + ', ' + b, FEATURE_COLUMNS)}
from credit.credit
'''
X = (
    pd
    .DataFrame(client.query(query).named_results())
    .assign(job = lambda x: x['job'].apply(lambda x: job_list[x]))
)

query = fr'''
select {TARGET_COLUMN_NAME}
from credit.credit
'''
y = pd.DataFrame(client.query(query).named_results())

df = (
    X
    .join(y)
)

# MLflow connection

In [20]:
warnings.filterwarnings('ignore')
mlflow.set_tracking_uri('http://localhost:5000/')
print("URI", mlflow.get_tracking_uri())
mlflow.set_experiment(f'credit_prediction_{datetime.now().strftime('01.%m.%Y')}')

URI http://localhost:5000/


2025/05/12 23:25:52 INFO mlflow.tracking.fluent: Experiment with name 'credit_prediction_01.05.2025' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow/1', creation_time=1747081552636, experiment_id='1', last_update_time=1747081552636, lifecycle_stage='active', name='credit_prediction_01.05.2025', tags={}>

# Model creation

In [22]:
numeric_transformer = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, COLUMNS_TO_SCALE),
        ("cat", categorical_transformer, COLUMNS_TO_ENCODE)
    ]
)

In [23]:
X_preproccessed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size = 0.25,
    random_state=42
)

### Optimize hyperparams (optuna)

In [ ]:
def objective(trial):    
    # CatBoostClassifier hyperparams
    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "used_ram_limit": "3gb",
    }

    # model
    estimator = CatBoostClassifier(**param, verbose=False)
    
    # pipelines
    clf_pipeline = Pipeline(steps = [
    ("preprocessor", preprocessor),
    ("classifier", estimator)
])
    # cross-validation accuracy counting
    accuracy = cross_val_score(clf_pipeline, df[FEATURE_COLUMNS], df[TARGET_COLUMN_NAME], cv=10, scoring= 'accuracy').mean()
    return accuracy

#study = optuna.create_study(direction="maximize", study_name="CBC-2023-01-14-14-30", storage='sqlite:///db/CBC-2023-01-14-14-30.db')
# study = optuna.create_study(direction="maximize", study_name=f"{datetime.now().strftime('%Y%m%d-%H%M%S')}")
study = optuna.create_study(direction="maximize", study_name="credit_classifier")

# Hyperparams searching
study.optimize(objective, n_trials=10)

# best result is
params = study.best_params
params

In [ ]:
catboostclassifier = Pipeline(steps = [
    ("preprocessor", preprocessor),
    ("classifier", CatBoostClassifier(**params, verbose=False))
])
catboostclassifier.fit(X_train, y_train)

## Wrap dataset with Giskard

In [ ]:
raw_data = pd.concat([X_test, y_test], axis = 1)
giskard_dataset = Dataset(
    df = raw_data,
    target=TARGET_COLUMN_NAME,
    name = "German credit scoring dataset",
    cat_columns=COLUMNS_TO_ENCODE
)

## Wrap model with Giskard

In [ ]:
giskard_model = Model(
    model=catboostclassifier,
    model_type="classification",     # Either regression, classification or text_generation.
    name="Chunk classification",
    classification_labels=catboostclassifier.classes_,  # Their order MUST be identical to the prediction_function's output order
    feature_names=FEATURE_COLUMNS     # Default: all columns of your dataset
)

## Scan model with Giskard

In [ ]:
results = scan(giskard_model, giskard_dataset, verbose=False)
results.to_html("giskard_scan_result.html")

# Minio

In [6]:
def s3_upload_model():
    # Create a client with the MinIO server playground, its access key
    # and secret key.
    client = Minio("s3:9099",
        access_key=ACCESS_KEY,
        secret_key=SECRET_KEY,
        secure=False
    )

    # The file to upload, change this path if needed
    source_file = "model.pkl"

    # The destination bucket and filename on the MinIO server
    bucket_name = "credit-model"
    destination_file = "model.pkl"

    # Make the bucket if it doesn't exist.
    found = client.bucket_exists(bucket_name)
    if not found:
        client.make_bucket(bucket_name)
        print("Created bucket", bucket_name)
    else:
        print("Bucket", bucket_name, "already exists")

    # Upload the file, renaming it in the process
    client.fput_object(
        bucket_name, destination_file, source_file,
    )
    print(
        source_file, "successfully uploaded as object",
        destination_file, "to bucket", bucket_name,
    )

try:
    s3_upload_model()
except S3Error as exc:
    print("error occurred.", exc)

Bucket credit-model already exists
model.pkl successfully uploaded as object model.pkl to bucket credit-model
